In [41]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import numpy as np

import os

from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression

## <font size="4"><b>Оглавление </b></p></font> 
<a id='contents'></a>

1. [Постановка задания](#intro)
2. [Обзор данных](#data_review)
- [Выводы](#data_review_summary)
3. [Преобразование данных](#data_modify)
- [Подготовка выборок](#data_modify_prepare)
- [Преобразование признаков](#data_modify_X)
- [Математическое обоснование](#data_modify_math_proof)
- [Выводы](#data_modify_summary)
4. [Построение моделей](#models)
5. [Итоги исследования](#summary)

<a id='intro'></a>

# **Защита персональных данных клиентов**

**Заказчик** —  страховая компания «Хоть потоп»

**Цель проекта** - проверить следующие гипотезы:
1. Разработать метод преобразования/шифрования данных персональной информации.

**Поставленные задачи**
1. Загрузить и изучите данные. 
2. Выполнить преобразование данных таким образом, чтобы качество модели машинного обучения на основе алгоритма «Линейная регрессия» не ухудшилось.

**Дальнейшее использование**

Результаты будут использоваться при дальнейшем шифровании персональных данных.

[К оглавлению](#contents)

<a id='data_review'></a>

## **I. Обзор данных**

Сначала составим представление о полученных исходных данных.

Прочитаем файл `insurance.csv` из папки `/datasets/` и сохраним его в переменной `df`:

In [2]:
pth1 = 'D:/Programs/Jupyter_projects/datasets/insurance.csv'
pth2 = '/datasets/insurance.csv'

if os.path.exists(pth1):
    df = pd.read_csv(pth1, sep=',')
elif os.path.exists(pth2):
    df = pd.read_csv(pth2, sep=',')
else:
    print('Something is wrong')

Выведем на экран первые пять строк таблицы:

In [3]:
display(df.head())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Получим общую информацию о таблице: <a id='df.info'></a>

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Итак, в таблице пять столбцов. Типы данных в столбцах — целые числа `int` и вещественные числа `float`.

Согласно документации к данным:

* Признаки: `пол`, `возраст` и `зарплата застрахованного`, `количество членов его семьи`.
* Целевой признак: `количество страховых выплат клиенту за последние 5 лет`.

Количество значений в столбцах не различается. Значит, в данных нет **пропущенных значений**.

Названия колонок написаны кириллицей с заглавной первой буквой.

<a id='data_review_summary'></a>

### **Выводы**

В каждой строке таблицы — данные об уникальном клиенте. Часть колонок описывает общие персональные параметры клиента: пол, дети, возраст, заработная плата, количество членов семьи. Остальные данные рассказывают о бизнес отношениях с банком: количество страховых выплат клиенту за последние 5 лет.

В данных не встречается пропусков.

[К оглавлению](#contents)

<a id='data_modify'></a>

## **II. Преобразование данных**

<a id='data_modify_prepare'></a>

### 1. Подготовка выборок

Сохраним наши признаки в соответствующие выборки

In [5]:
features = df.drop(columns = ['Страховые выплаты'],axis = 1)
target = df['Страховые выплаты']

In [6]:
features.head()

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


[К оглавлению](#contents)

<a id='data_modify_X'></a>

### 2. Преобразование признаков

Предоставленные данные представляют собой один единый датафрейм, разделения на обучающую и тестовую выборки у нас сейчас нет. Поэтому выполним это сейчас с помощью алгоритма `train_test_split`.

In [14]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=.25, random_state=12345)

Для преобразования персональных данных домножим все признаки из `features` на обратимую матрицу. Для этого сначала необходимо создавать случайную матрицу до тех пор, пока она не будет обратимой, во избежание маловероятного появления необратимой матрицы.

Для этого напишем **функцию `X_modified`**, которая внутри себя создает случайную обратимую квадратную матрицу и домнажает на нее наши признаки. Функция возвращает преобразованные признаки и обратную матрицу для P, чтобы потом можно было обратно восстановить исходные признаки.

In [15]:
def X_modified(X, X_test, seed):
    try:
        state = np.random.RandomState(seed=seed)
        # Квадратная матрица P
        rand_matrix = state.normal(size=(X.shape[1], X.shape[1]))
        # Проверка на обратимость
        inv = np.linalg.inv(rand_matrix)
        # Умножаем признаки на обратимую матрицу
        x_modified = X @ rand_matrix
        x_test_modified = X_test @ rand_matrix
        return x_modified, x_test_modified, inv
    
    except np.linalg.LinAlgError:
        # Если возникла ошибка, заново запускаем нашу функцию
        X_modified(X, X_test, seed)

Примерим созданную функцию для преобразования данных и записи в новую переменную. 

In [16]:
features_new_train, features_new_test, inv_P = X_modified(features_train, features_test, 12345)

[К оглавлению](#contents)

<a id='data_modify_math_proof'></a>

### 3. Матемарическое обоснование использования обратимой матрицы для шифрования

Докажем принятое решение о возможности шифрования данных путем домножения их на обратимую матрицу. Для доказательства воспользуемся формулой задачи обучения линейной регрессии для функции потерь MSE:
$$w = argmin_w MSE (Xw,y)$$

При этом минимальное значение MSE достигается, когда веса равны величине:
$$w = (X^{T} X)^{-1} X^{T} y $$

Нам также понадобятся:
- Правило умножения на обратную матрицу с полуением единичной:
$$ X X^{-1} = X^{-1} X = E $$

- Правило умножения на единичную матрицу:
$$ X E = E X = X $$

Тогда домножим исходные признаки на обратимую матрицу `P` и раскроем все скобки:
$$w` = ((XP)^{T} XP)^{-1} (XP)^{T} y  = (P^{T} X^{T} XP)^{-1} P^{T} X^{T} y = P^{-1} (X X^{T})^{-1} (P^{T})^{-1} P^{T} X^{T} y = P^{-1} w $$
$$w` = P^{-1} w $$

Получается, что для обратного получения исходных признаков нам необходимо будет домножить на матрицу, обратную той, что была использована в начале (хранится в переменной `inv_P`) 

<a id='data_modify_summary'></a>

#### **Выводы**

Во время преобразования данных были сделаны следующие выводы:
* В качестве шифрования/преобразования персональных данных клиентов может использоваться произведение признаков на случайно полученную обратимую матрицу.
* Чтобы потом дешифровать данные достаточно будет домножить признаки на обратную матрицу той, которая использовалась при шифровании.

Осталось доказать данные вычисления экспериментально, построив модель машинного обучения на основе алгоритма «Линейная регрессия» с разными обучающими выборками (зашифрованными данными и незашифрованными) и сравнив качество: если оно не ухудшилось, то данный способ преобразования нам подходит.

[К оглавлению](#contents)

<a id='models'></a>

## **III. Построение моделей**

### 1. Модель «Линейная регрессия» для исходных данных

Построим модель линейной регрессии и проверим ее качество на преобразованных и непреобразованных данных. Показателем качества выберем R2 score

In [36]:
def model_LR_for_check_r2_score(features_train, features_test):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_test)
    R2_score = r2_score(target_test, predictions)
    return R2_score

In [37]:
R2_score_origin = model_LR_for_check_r2_score(features_train, features_test)

R2_score_modified = model_LR_for_check_r2_score(features_new_train, features_new_test)

In [40]:
print('\n\033[4m\033[1m\033[31m{}\033[0m'.format('Модель «Линейная регрессия» на исходных данных:'))
print('R2 score:', R2_score_origin)

print('\n\033[4m\033[1m\033[31m{}\033[0m'.format('Модель «Линейная регрессия» на преобразованных данных:'))
print('R2 score:', R2_score_modified)

print('\n\033[4m\033[1m\033[31m{}\033[0m'.format('Различия в качестве составляет:'), abs(R2_score_origin - R2_score_modified) / R2_score_origin)


Модель «Линейная регрессия» на исходных данных:
R2 score: 0.44414121885134106

Модель «Линейная регрессия» на преобразованных данных:
R2 score: 0.44414121885148816

Различия в качестве составляет: 3.312112105768565e-13


Как мы видим, различия в качестве модели на основании исходных и преобразованных данных на уровне погрешности, поэтому можно считать, что использовать в качестве шифрования домножение на случайно полученную обратимую матрицу возможно без потерь качества предсказания модели машинного обучения. В последующем извлечение необходимых данных можно выполнять с помощью обратной использованной матрицы. 

Например, возьмем 5 первых объектов (клиентов) исходных данных

In [ ]:
features.head()

И их же только в преобразованных данных

In [ ]:
features_new.head()

Выполним обратное преобразование, домножив на обратную матрицу

In [ ]:
vice_versa = round(features_new.head() @ inv_P)
vice_versa.columns = features.columns
display(vice_versa)

<a id='summary'></a>

## **V. Итоги исследования**

На основании предоставленных данных мы
- выбрали метод шифрования/дешифрования персональных данных через домножение первых на обратимую случайную матрицу,
- обосновали свой выбор через математические формулы операций с матрицами, определение линейной регрессии через минимизацию MAE, 
- доказали возможность его использования с помощью построения модели машинного обучения по исходным и преобразованным данным с последующим сравнением качества R2 score. 

[К оглавлению](#contents)